# Basics
The basic notebook just imports the required dependencies and sets the ports and addresses correctly to be accessible outside of the Docker container. It then creates a link for viewing the data

In [1]:
from __future__ import print_function
from neuroglancer.server import global_server_args
# setup the binding so it is visible outside the container
global_server_args['bind_port'] = 8989
global_server_args['bind_address'] = '00.0.0.0' # TODO: the server renames addresses of 0.0.0.0 and they become unusable
import neuroglancer
import numpy as np

# Synthetic Data
Since it is much easier to generate synthetic data than copy full real images a 3D image sized 100x100x100 is greated and then colored using a simple sinusoidal relationship

In [3]:
a = np.zeros((3, 100, 100, 100), dtype=np.uint8)
ix, iy, iz = np.meshgrid(*[np.linspace(0, 1, n) for n in a.shape[1:]], indexing='ij')
a[0, :, :, :] = np.abs(np.sin(4 * (ix + iy))) * 255
a[1, :, :, :] = np.abs(np.sin(4 * (iy + iz))) * 255
a[2, :, :, :] = np.abs(np.sin(4 * (ix + iz))) * 255

b = np.cast[np.uint32](np.floor(np.sqrt((ix - 0.5)**2 + (iy - 0.5)**2 + (iz - 0.5)**2) * 10))
b = np.pad(b, 1, 'constant')
print(a.shape, b.shape)

(3, 100, 100, 100) (102, 102, 102)


# Create the viewer
Here the viewer is created with a simple shader to make the rendering colorful

In [2]:
neuroglancer.set_static_content_source(url='https://neuroglancer-demo.appspot.com')

viewer = neuroglancer.Viewer(voxel_size=[10, 10, 10])
viewer.add(a,
           name='a',
           # offset is in nm, not voxels
           offset=(200, 300, 150),
           shader="""
void main() {
  emitRGB(vec3(toNormalized(getDataValue(0)),
               toNormalized(getDataValue(1)),
               toNormalized(getDataValue(2))));
}
""")
viewer.add(b, name='b')

# Show the Results
Here when we show the viewer we get a link, altnatively the results can be shown using the IPython HTML displaying interface

In [13]:
viewer

http://00.0.0.0:8989/static/6a71f7503d6f329832065b243340becfa9ffcd90/#!%7B%22layers%22:%7B%22a%22:%7B%22source%22:%22python://http://00.0.0.0:8989/c6554b7a46fd1731f2f3cee1975697e457492e01%22,%22type%22:%22image%22,%22shader%22:%22%5Cnvoid%20main()%20%7B%5Cn%20%20emitRGB(vec3(toNormalized(getDataValue(0)),%5Cn%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20toNormalized(getDataValue(1)),%5Cn%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20toNormalized(getDataValue(2))));%5Cn%7D%5Cn%22%7D,%22b%22:%7B%22source%22:%22python://http://00.0.0.0:8989/587332423f55d75292ab945b90512f3b63987adf%22,%22type%22:%22segmentation%22%7D%7D,%22navigation%22:%7B%22pose%22:%7B%22voxelSize%22:%5B10,10,10%5D%7D%7D%7D

In [15]:
from IPython.display import HTML
HTML(url = str(viewer))